In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('pfm_train _digitization2.csv')
df.head()
# rename some columns
df.rename(columns={'Attrition': 'left'}, inplace=True)


In [2]:
X = df.drop('left', axis=1)
y = df['left']
# Split Training Set from Testing Set (70/30)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=10)

In [3]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
# transform our training features
X_train_std = stdsc.fit_transform(X_train)
#print (X_train_std[0])
# transform the testing features in the same way
X_test_std = stdsc.transform(X_test)

In [4]:
# Cross validation
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=5, test_size=0.3,random_state=10)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_param_1 = {'n_estimators': range(1,100)}
rf_grid_1 = GridSearchCV( RandomForestClassifier(random_state=10),param_grid=rf_param_1, cv=cv)
rf_grid_1.fit(X_train_std, y_train)
print('Parameter with best score:')
print(rf_grid_1.best_params_)
print('Cross validation score:', rf_grid_1.best_score_)

Parameter with best score:
{'n_estimators': 29}
Cross validation score: 0.856277056277


In [27]:
rf_param_2 = {'max_depth':range(3,29,1)}
rf_grid_2 = GridSearchCV( RandomForestClassifier(n_estimators= 29,random_state=10), 
                         param_grid = rf_param_2 ,cv=cv)
rf_grid_2.fit(X_train_std, y_train)
print('Parameter with best score:')
print(rf_grid_2.best_params_)
print('Cross validation score:', rf_grid_2.best_score_)

Parameter with best score:
{'max_depth': 13}
Cross validation score: 0.857142857143


In [29]:
rf_param_4 = {'max_features':range(3,15,1)}
rf_grid_4 = GridSearchCV( RandomForestClassifier(n_estimators=29,random_state=13,max_depth=13),
                         param_grid = rf_param_4 ,cv=cv)
rf_grid_4.fit(X_train_std, y_train)
print('Parameter with best score:')
print(rf_grid_4.best_params_)
print('Cross validation score:', rf_grid_4.best_score_)

Parameter with best score:
{'max_features': 12}
Cross validation score: 0.861471861472


In [21]:
best_rf = rf_grid_2.best_estimator_
print('Test score:', best_rf.score(X_test_std, y_test))
#best_rf.fit(X_test_std, y_test)
print(best_rf.predict(X_test_std))
best_rf 

Test score: 0.842424242424
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features=11, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=29, n_jobs=1,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

In [ ]:
df_1=pd.read_csv('pfm_test_digitization2.csv')
std_df_1=stdsc.transform(df_1)
answer=best_rf.predict(std_df_1)

import csv  
def storFile(data,fileName):  
    data = list(map(lambda x:[x],data))  
    with open(fileName,'w',newline ='') as f:  
        mywrite = csv.writer(f)  
        for i in data:  
            mywrite.writerow(i)  
storFile(answer,'pfm_test10_n=29_f=11_d=16_n_splits=5_entropy.csv')  
print(answer)

In [25]:
# feature importance scores
features = X.columns
feature_importances = best_rf.feature_importances_

features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort_values('Importance Score', inplace=True, ascending=False)

features_df

,Features,Importance Score
13,MonthlyIncome,0.124354
15,OverTime,0.079812
0,Age,0.075484
3,DistanceFromHome,0.064100
16,PercentSalaryHike,0.052695
20,TotalWorkingYears,0.052428
14,NumCompaniesWorked,0.048947
19,StockOptionLevel,0.035801
11,JobSatisfaction,0.035391
6,EnvironmentSatisfaction,0.034044


## 